## Multiple Year Performance By Country

In [ ]:
multi_year_performance_by_country_df = spark.sql("""WITH CountryYearlyMedals AS (
    SELECT
        c.CountryName,
        YEAR(e.Date) AS Year,
        COUNT(*) AS TotalMedals
    FROM
        global_temp.medal_fact m
    JOIN
        global_temp.athlete_dimension a ON m.AthleteID = a.AthleteID
    JOIN
        global_temp.event_dimension e ON m.EventID = e.EventID
    JOIN
        global_temp.country_dimension c ON a.CountryID = c.CountryID
    GROUP BY
        c.CountryName, YEAR(e.Date)
),
CountryYearlyMedalsLag AS (
    SELECT
        CountryName,
        Year,
        TotalMedals,
        LAG(TotalMedals, 1) OVER (PARTITION BY CountryName ORDER BY Year) AS PreviousYearMedals
    FROM
        CountryYearlyMedals
)
SELECT
    CountryName,
    Year,
    TotalMedals,
    PreviousYearMedals,
    (TotalMedals - PreviousYearMedals) AS YearOverYearChange
FROM
    CountryYearlyMedalsLag
ORDER BY
    CountryName, Year""")
multi_year_performance_by_country_df.display()

## Country Performance

In [ ]:
country_performance_df = spark.sql("""WITH CountrySportPerformance AS (
    SELECT
        c.CountryName,
        s.SportName,
        COUNT(m.MedalFactID) AS TotalMedals,
        RANK() OVER (PARTITION BY s.SportName ORDER BY COUNT(m.MedalFactID) DESC) AS SportRank
    FROM
        global_temp.medal_fact m
    JOIN
        global_temp.athlete_dimension a ON m.AthleteID = a.AthleteID
    JOIN
        global_temp.event_dimension e ON m.EventID = e.EventID
    JOIN
        global_temp.sport_dimension s ON e.SportID = s.SportID
    JOIN
        global_temp.country_dimension c ON a.CountryID = c.CountryID
    GROUP BY
        c.CountryName, s.SportName
)
SELECT
    CountryName,
    SportName,
    TotalMedals,
    SportRank
FROM
    CountrySportPerformance
ORDER BY
    SportName, SportRank;
""")

country_performance_df.display()

## Athlete Career

In [ ]:
athlete_career_df = spark.sql("""WITH AthleteCareer AS (
    SELECT
        a.Name AS AthleteName,
        c.CountryName AS CountryName,
        MIN(YEAR(p.Date)) AS FirstYear,
        MAX(YEAR(p.Date)) AS LastYear,
        COUNT(DISTINCT YEAR(p.Date)) AS ActiveYears
    FROM
        global_temp.athlete_performance_fact p
    JOIN
        global_temp.athlete_dimension a ON p.AthleteID = a.AthleteID
    JOIN
        global_temp.country_dimension c ON a.CountryID = c.CountryID
    GROUP BY
        a.Name, c.CountryName
)
SELECT
    AthleteName,
    CountryName,
    FirstYear,
    LastYear,
    ActiveYears
FROM
    AthleteCareer
ORDER BY
    ActiveYears DESC;
""")
athlete_career_df.display()

## Venue Utilization

In [ ]:
venue_utilization_df = spark.sql("""SELECT
    v.VenueName,
    YEAR(e.Date) AS Year,
    ROUND(AVG(a.Attendance / v.Capacity) * 100, 2) AS AvgUtilizationPercentage
FROM
    global_temp.event_dimension e
JOIN
    global_temp.venue_dimension v ON e.VenueID = v.VenueID
JOIN
    global_temp.attendance_fact a ON e.EventID = a.EventID
GROUP BY
    v.VenueName, YEAR(e.Date)
ORDER BY
    v.VenueName, Year;
""")
venue_utilization_df.display()

In [ ]:
medal_fact_df = spark.sql("""SELECT * FROM global_temp.medal_fact WHERE MedalType IN ('Gold', 'Silver', 'Bronze')""")
medal_fact_df.createOrReplaceGlobalTempView("medal_fact")

## Medals By Country

In [ ]:
gold_medal_by_continent_df = spark.sql("""WITH MedalByContinent AS (
    SELECT
        c.Continent,
        m.MedalType,
        COUNT(*) AS MedalCount
    FROM
        global_temp.medal_fact m
    JOIN
        global_temp.athlete_dimension a ON m.AthleteID = a.AthleteID
    JOIN
        global_temp.country_dimension co ON a.CountryID = co.CountryID
    JOIN
        (SELECT
            CountryID,
            CASE
                WHEN CountryName IN ('USA', 'Canada', 'Mexico') THEN 'North America'
                WHEN CountryName IN ('Brazil', 'Argentina') THEN 'South America'
                WHEN CountryName IN ('China', 'Japan', 'India') THEN 'Asia'
                WHEN CountryName IN ('Australia') THEN 'Australia'
                WHEN CountryName IN ('UK', 'Germany', 'France') THEN 'Europe'
                ELSE 'Other'
            END AS Continent
        FROM
            global_temp.country_dimension) c ON co.CountryID = c.CountryID
    GROUP BY
        c.Continent, m.MedalType
)
SELECT
    Continent,
    MedalType,
    MedalCount
FROM
    MedalByContinent
ORDER BY
    Continent, MedalType;
""")
gold_medal_by_continent_df.display()